In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
import random

# Bayesian model imports
import pystan
from scipy.special import expit as logistic_sigmoid

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Exploratory Data Analysis (EDA)
print("Number of training samples:", x_train.shape[0])
print("Number of test samples:", x_test.shape[0])
print("Shape of an image:", x_train[0].shape)

# Visualize some samples from the dataset
fig, axes = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    axes[i].imshow(x_train[i], cmap='gray')
    axes[i].axis('off')
plt.show()

# Data preprocessing
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1, 28*28) / 255.0

# Split validation set from training set
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

# Train three different models

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(x_train, y_train)
log_reg_val_pred = log_reg.predict(x_val)
log_reg_accuracy = accuracy_score(y_val, log_reg_val_pred)

# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(x_train, y_train)
rf_val_pred = rf_clf.predict(x_val)
rf_accuracy = accuracy_score(y_val, rf_val_pred)

# Neural Network
mlp_clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=20)
mlp_clf.fit(x_train, y_train)
mlp_val_pred = mlp_clf.predict(x_val)
mlp_accuracy = accuracy_score(y_val, mlp_val_pred)

# Bayesian Logistic Regression Model using PyStan
stan_model_code = """
data {
  int<lower=0> N;  // number of observations
  int<lower=0> K;  // number of predictors
  matrix[N, K] X;  // predictor matrix
  int<lower=0, upper=1> y[N];  // outcome variable
}
parameters {
  vector[K] beta;  // coefficients for predictors
}
model {
  y ~ bernoulli_logit(X * beta);  // likelihood
  beta ~ normal(0, 1);  // prior
}
"""

stan_data = {
    'N': x_train.shape[0],
    'K': x_train.shape[1],
    'X': x_train,
    'y': y_train
}

stan_model = pystan.StanModel(model_code=stan_model_code)
stan_fit = stan_model.sampling(data=stan_data, iter=1000, chains=4)

# Extract posterior means for beta
beta_post = stan_fit.extract()['beta']
beta_mean = np.mean(beta_post, axis=0)

# Make predictions using the Bayesian model
def bayesian_predict(X, beta_mean):
    return logistic_sigmoid(np.dot(X, beta_mean))

bayesian_val_pred = bayesian_predict(x_val, beta_mean)
bayesian_val_pred_class = (bayesian_val_pred > 0.5).astype(int)
bayesian_accuracy = accuracy_score(y_val, bayesian_val_pred_class)

# Output validation accuracy for each model
print(f"Logistic Regression Accuracy: {log_reg_accuracy}")
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Neural Network Accuracy: {mlp_accuracy}")
print(f"Bayesian Model Accuracy: {bayesian_accuracy}")

# Define a function to evaluate a chosen model
def evaluate_model(model_name, n_samples):
    model_dict = {
        "logistic_regression": log_reg,
        "random_forest": rf_clf,
        "neural_network": mlp_clf,
        "bayesian_model": (bayesian_predict, beta_mean)
    }
    
    if model_name not in model_dict:
        print("Invalid model name.")
        return
    
    indices = random.sample(range(len(x_test)), n_samples)
    x_sample = x_test[indices]
    y_sample = y_test[indices]
    
    if model_name == "bayesian_model":
        y_pred = model_dict[model_name][0](x_sample, model_dict[model_name][1])
        y_pred = (y_pred > 0.5).astype(int)
    else:
        model = model_dict[model_name]
        y_pred = model.predict(x_sample)
    
    results = pd.DataFrame({
        "Actual": y_sample,
        "Predicted": y_pred
    })
    
    accuracy_per_digit = results.groupby("Actual").apply(lambda x: (x["Actual"] == x["Predicted"]).mean())
    
    print(f"Accuracy per digit for {model_name}:")
    print(accuracy_per_digit)
    
    return results

# Interactive program for user to evaluate models
def interactive_evaluation():
    while True:
        model_name = input("Choose a model (logistic_regression, random_forest, neural_network, bayesian_model): ")
        n_samples = int(input("Enter the number of samples to evaluate: "))
        evaluate_model(model_name, n_samples)
        
        cont = input("Do you want to continue? (yes/no): ")
        if cont.lower() != "yes":
            break

# Run the interactive evaluation
interactive_evaluation()
